In [ ]:
!git clone https://github.com/pytorch/fairseq -q

In [ ]:
!pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio===0.9.1 -f  https://download.pytorch.org/whl/torch_stable.html 

In [ ]:
%cd fairseq
!pip uninstall numpy -q -y
!pip install --editable ./ -q
%cd ..

In [ ]:
!pip install sentencepiece
!pip install tensorboardX
!pip install pandas
!pip install pymysql

In [ ]:
!wget "http://dl.fbaipublicfiles.com/m2m_100/spm.128k.model"
!wget "http://dl.fbaipublicfiles.com/m2m_100/data_dict.128k.txt"
!wget "http://dl.fbaipublicfiles.com/m2m_100/model_dict.128k.txt"
!wget "http://dl.fbaipublicfiles.com/m2m_100/language_pairs_small_models.txt"

In [ ]:
!wget "http://chuangshiqingyu.com/model/418M_53000_20211005_best.pt"

In [ ]:
import pymysql
conn = pymysql.connect(
    host="127.0.0.1",
    user="root",password="root",
    database="metateam",
    charset="utf8",cursorclass=pymysql.cursors.DictCursor)

In [ ]:
import pandas as pd

def do_translate():
    !python fairseq/scripts/spm_encode.py \
            --model spm.128k.model \
            --output_format=piece \
            --inputs=ko_txt.txt \
            --outputs=ko_txt.ko

    !fairseq-preprocess \
        --source-lang ko --target-lang en \
        --only-source	\
        --testpref ko_txt \
        --thresholdsrc 0 --thresholdtgt 0 \
        --destdir data_bin \
        --srcdict data_dict.128k.txt --tgtdict data_dict.128k.txt

    !fairseq-generate "data_bin/"  --batch-size 32 \
          --path "418M_53000_20211005_best.pt" \
          --fixed-dictionary model_dict.128k.txt \
          -s ko -t en \
          --remove-bpe sentencepiece \
          --beam 5 \
          --task translation_multi_simple_epoch \
          --lang-pairs language_pairs_small_models.txt \
          --decoder-langtok \
          --encoder-langtok src \
          --gen-subset test > gen_out

    !grep ^H gen_out | cut -f3- > gen.out.sys
    !grep ^T gen_out | cut -f2- > gen.out.ref
    !grep ^S gen_out | cut -f2- > gen.out.src


    output_text = open("gen.out.sys", "r").read().split("\n")

    print(output_text)


In [ ]:
def read_result():
    import re

    line_count = 0
    translate_sentence_list = []
    with open('./gen_out') as fd:
        while True:
            line = fd.readline()
            line_count = line_count + 1
            line_list = line.split("	")
            if not line:
                break
            if (len(line_list)>2 and line_list[0][:1] == "H"):
                line_list[0] = int(re.findall(r'\d+', line_list[0])[0])
                line_list[2] = line_list[2].encode("utf-8"). decode("utf-8")
                translate_sentence_list.append(line_list)
    translate_sentence_list.sort()
    return translate_sentence_list


In [ ]:
def sendMail(receiver_email,link):
    import smtplib, ssl
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart

    sender_email = "" #change 
    password = "" #change 

    message = MIMEMultipart("alternative")
    message["Subject"] = "META Translated"
    message["From"] = sender_email
    message["To"] = receiver_email

    # Create the plain-text and HTML version of your message

    html = """\
    <!doctype html>
    <html>
      <head>
        <meta name="viewport" content="width=device-width">
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
        <title>Simple Transactional Email</title>
        <style>
        /* -------------------------------------
            INLINED WITH htmlemail.io/inline
        ------------------------------------- */
        /* -------------------------------------
            RESPONSIVE AND MOBILE FRIENDLY STYLES
        ------------------------------------- */
        @media only screen and (max-width: 620px) {
          table[class=body] h1 {
            font-size: 28px !important;
            margin-bottom: 10px !important;
          }
          table[class=body] p,
                table[class=body] ul,
                table[class=body] ol,
                table[class=body] td,
                table[class=body] span,
                table[class=body] a {
            font-size: 16px !important;
          }
          table[class=body] .wrapper,
                table[class=body] .article {
            padding: 10px !important;
          }
          table[class=body] .content {
            padding: 0 !important;
          }
          table[class=body] .container {
            padding: 0 !important;
            width: 100% !important;
          }
          table[class=body] .main {
            border-left-width: 0 !important;
            border-radius: 0 !important;
            border-right-width: 0 !important;
          }
          table[class=body] .btn table {
            width: 100% !important;
          }
          table[class=body] .btn a {
            width: 100% !important;
          }
          table[class=body] .img-responsive {
            height: auto !important;
            max-width: 100% !important;
            width: auto !important;
          }
        }

        /* -------------------------------------
            PRESERVE THESE STYLES IN THE HEAD
        ------------------------------------- */
        @media all {
          .ExternalClass {
            width: 100%;
          }
          .ExternalClass,
                .ExternalClass p,
                .ExternalClass span,
                .ExternalClass font,
                .ExternalClass td,
                .ExternalClass div {
            line-height: 100%;
          }
          .apple-link a {
            color: inherit !important;
            font-family: inherit !important;
            font-size: inherit !important;
            font-weight: inherit !important;
            line-height: inherit !important;
            text-decoration: none !important;
          }
          #MessageViewBody a {
            color: inherit;
            text-decoration: none;
            font-size: inherit;
            font-family: inherit;
            font-weight: inherit;
            line-height: inherit;
          }
          .btn-primary table td:hover {
            background-color: #34495e !important;
          }
          .btn-primary a:hover {
            background-color: #34495e !important;
            border-color: #34495e !important;
          }
        }
        </style>
      </head>
      <body class="" style="background-color: #f6f6f6; font-family: sans-serif; -webkit-font-smoothing: antialiased; font-size: 14px; line-height: 1.4; margin: 0; padding: 0; -ms-text-size-adjust: 100%; -webkit-text-size-adjust: 100%;">
        <span class="preheader" style="color: transparent; display: none; height: 0; max-height: 0; max-width: 0; opacity: 0; overflow: hidden; mso-hide: all; visibility: hidden; width: 0;">This is preheader text. Some clients will show this text as a preview.</span>
        <table role="presentation" border="0" cellpadding="0" cellspacing="0" class="body" style="border-collapse: separate; mso-table-lspace: 0pt; mso-table-rspace: 0pt; width: 100%; background-color: #f6f6f6;">
          <tr>
            <td style="font-family: sans-serif; font-size: 14px; vertical-align: top;">&nbsp;</td>
            <td class="container" style="font-family: sans-serif; font-size: 14px; vertical-align: top; display: block; Margin: 0 auto; max-width: 580px; padding: 10px; width: 580px;">
              <div class="content" style="box-sizing: border-box; display: block; Margin: 0 auto; max-width: 580px; padding: 10px;">

                <!-- START CENTERED WHITE CONTAINER -->
                <table role="presentation" class="main" style="border-collapse: separate; mso-table-lspace: 0pt; mso-table-rspace: 0pt; width: 100%; background: #ffffff; border-radius: 3px;">

                  <!-- START MAIN CONTENT AREA -->
                  <tr>
                    <td class="wrapper" style="font-family: sans-serif; font-size: 14px; vertical-align: top; box-sizing: border-box; padding: 20px;">
                      <table role="presentation" border="0" cellpadding="0" cellspacing="0" style="border-collapse: separate; mso-table-lspace: 0pt; mso-table-rspace: 0pt; width: 100%;">
                        <tr>
                          <td style="font-family: sans-serif; font-size: 14px; vertical-align: top;">
                            <p style="font-family: sans-serif; font-size: 14px; font-weight: normal; margin: 0; Margin-bottom: 15px;">Hi there,</p>
                            <p style="font-family: sans-serif; font-size: 14px; font-weight: normal; margin: 0; Margin-bottom: 15px;">We have translated the document for you. Please check it.</p>
                            <table role="presentation" border="0" cellpadding="0" cellspacing="0" class="btn btn-primary" style="border-collapse: separate; mso-table-lspace: 0pt; mso-table-rspace: 0pt; width: 100%; box-sizing: border-box;">
                              <tbody>
                                <tr>
                                  <td align="left" style="font-family: sans-serif; font-size: 14px; vertical-align: top; padding-bottom: 15px;">
                                    <table role="presentation" border="0" cellpadding="0" cellspacing="0" style="border-collapse: separate; mso-table-lspace: 0pt; mso-table-rspace: 0pt; width: auto;">
                                      <tbody>
                                        <tr>
                                          <td style="font-family: sans-serif; font-size: 14px; vertical-align: top; background-color: #3498db; border-radius: 5px; text-align: center;"> <a href="""+link+""" target="_blank" style="display: inline-block; color: #ffffff; background-color: #3498db; border: solid 1px #3498db; border-radius: 5px; box-sizing: border-box; cursor: pointer; text-decoration: none; font-size: 14px; font-weight: bold; margin: 0; padding: 12px 25px; text-transform: capitalize; border-color: #3498db;">Download document</a> </td>
                                        </tr>
                                      </tbody>
                                    </table>
                                  </td>
                                </tr>
                              </tbody>
                            </table>
                            <p style="font-family: sans-serif; font-size: 14px; font-weight: normal; margin: 0; Margin-bottom: 15px;">If you have any questions, please contact us
    </p>
                          </td>
                        </tr>
                      </table>
                    </td>
                  </tr>

                <!-- END MAIN CONTENT AREA -->
                </table>

                <!-- START FOOTER -->
                <div class="footer" style="clear: both; Margin-top: 10px; text-align: center; width: 100%;">
                  <table role="presentation" border="0" cellpadding="0" cellspacing="0" style="border-collapse: separate; mso-table-lspace: 0pt; mso-table-rspace: 0pt; width: 100%;">
                    <tr>
                      <td class="content-block" style="font-family: sans-serif; vertical-align: top; padding-bottom: 10px; padding-top: 10px; font-size: 12px; color: #999999; text-align: center;">
                        <span class="apple-link" style="color: #999999; font-size: 12px; text-align: center;">Pusan National University</span>
                        <br> Nature Research Building 316, AI Lab</a>.
                      </td>
                    </tr>
                    <tr>
                      <td class="content-block powered-by" style="font-family: sans-serif; vertical-align: top; padding-bottom: 10px; padding-top: 10px; font-size: 12px; color: #999999; text-align: center;">
                        META TEAM</a>
                      </td>
                    </tr>
                  </table>
                </div>
                <!-- END FOOTER -->

              <!-- END CENTERED WHITE CONTAINER -->
              </div>
            </td>
            <td style="font-family: sans-serif; font-size: 14px; vertical-align: top;">&nbsp;</td>
          </tr>
        </table>
      </body>
    </html>
    """

    # Turn these into plain/html MIMEText objects

    part2 = MIMEText(html, "html")

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first

    message.attach(part2)

    # Create secure connection with server and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(
            sender_email, receiver_email, message.as_string()
        )

In [ ]:


def get_task():
    global conn
    cursor = conn.cursor()
    ids  = []
    cursor.execute("SELECT * FROM `app_translate_task` where status = 'notprocessed' limit 1")
    result = cursor.fetchall()
    for info in result:
        return info

def get_sentence(task):
    file = open("ko_txt.txt", "w",encoding='utf-8')
    
    global conn
    cursor = conn.cursor()
    ids  = []
    cursor.execute("SELECT * FROM `app_translate_sentence` where task_id = '"+str(task['id'])+"' ")
    result = cursor.fetchall()
    for info in result:
        file.write(info["sentence"]+"\n")
    file.close()

def filterText(text):
    text = text.replace("‘","\"")
    text = text.replace("'","\"")
    return text

def result_insert_to_database(task,sentence_result):
    global conn
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM `app_translate_sentence` where task_id = '"+str(task['id'])+"' ")
    result = cursor.fetchall()
    count = 0
    for info in result:
        sentence_result[count][2] = filterText(sentence_result[count][2])
        sentence_result[count][2] = sentence_result[count][2][2:-2]
        print(info)
        print(sentence_result[count][2])

        cursor.execute("UPDATE `app_translate_sentence` SET `translated_sentence` = '"+str(sentence_result[count][2])+"',status = 'processed'  WHERE `id` = '"+str(info['id'])+"'")
        count = count + 1
        
    cursor.execute("UPDATE `app_translate_task` SET status = 'processed'  WHERE `id` = '"+str(task['id'])+"'")
    

new_task = get_task()
if (new_task!=None):
    get_sentence(new_task)
    do_translate()
    translate_result = read_result()
    result_insert_to_database(new_task,translate_result)
    sendMail(new_task['email'],"http://metateam.site/customer/customerPageView?seccode="+str(new_task['seccode']))

